In [4]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "0riginal"

# Connect to database via CRUD Module
db = AnimalShelter(username, password, "nv-desktop-services.apporto.com", 32428, "AAC", "animals")

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
if '_id' in df.columns:
    df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'gslogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS 340 Dashboard'))),
    html.Center(html.I(html.H1('by Aaron Adeyemi'))),
    html.Hr(),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Div(
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        dcc.RadioItems(
            id = 'filter-type',
            options = [
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mount'},
                {'label': 'Disaster Rescue and Individual Tracking', 'value': 'disa'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value = 'reset'
        )
            
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        editable=False,
        sort_action="native",
        sort_mode="multi",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



#Radio button filters 
@app.callback([Output('datatable-id','data'),
               Output('datatable-id', 'columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
## FIX ME Add code to filter interactive data table with MongoDB queries
        # Water Rescue
        if filter_type == 'water':
            df = pd.DataFrame.from_records(db.read({
                'animal_type': 'Dog',
                'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
                'sex_upon_outcome': 'Intact Female',
                'age_upon_outcome_in_weeks': {'$gte':26.0, '$lte':156.0}
            }))
        
        # Mountain or Wilderness Rescue
        elif filter_type == 'mount':
            df = pd.DataFrame.from_records(db.read({
                'animal_type': 'Dog',
                'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 
                                  'Siberian Husky', 'Rottweiler']},
                'sex_upon_outcome': 'Intact Male',
                'age_upon_outcome_in_weeks': {'$gte':26.0, '$lte':156.0}
            }))
        
        #Disaster or Individual Tracking Rescue
        elif filter_type == 'disa':
            df = pd.DataFrame.from_records(db.read({
                'animal_type': 'Dog',
                'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 
                                  'Bloodhound', 'Rottweiler']},
                'sex_upon_outcome': 'Intact Male',
                'age_upon_outcome_in_weeks': {'$gte':20.0, '$lte':300.0}
                
            }))
        
        #Reset Scenario
        elif filter_type == 'reset':
            df = pd.DataFrame.from_records(db.read({}))
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
       
       
        return (data,columns)

# Highlight selected row
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns = [0]):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(viewData)
    
    return [
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
            #figure = px.histogram(df, x = 'Animal Breed')
        )    
    ]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, row_ids):
    dff = pd.DataFrame.from_dict(viewData)
    # Sets first num in row_ids array to the last selected row ID
    row_ids[0] = row_ids[len(row_ids) - 1]
    return [
        # row_ids[0] gets last selected row from DataTable and parameters 13 and 14 are long and lat of rows
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[row_ids[0],13],dff.iloc[row_ids[0],14]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[row_ids[0],13],dff.iloc[row_ids[0],14]], children=[
                # parameter num 4 gives the breed of animal
                dl.Tooltip(dff.iloc[row_ids[0],4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    # parameter 9 gives animal's name
                    html.P(dff.iloc[row_ids[0],9])
                ])
            ])
        ])
    ]


suppress_callback_exceptions=True
app.run_server(debug=True)


Dash app running on http://127.0.0.1:11723/
